In [2]:
import hashlib
import numpy as np
import pandas as pd

In [4]:
in_path  = "dataset/ews_customer_feedback.parquet"
out_path = "dataset/customer_feedback_text/ews_customer_feedback_with_text.parquet"

In [6]:
fb = pd.read_parquet(in_path).copy()

def _rng_for_row(row):
    # Stable seed from customer + cluster + last_offline_ts (if present)
    key = f"{row.get('customer_id','?')}|{row.get('cluster_id','?')}|{row.get('last_offline_ts','?')}"
    return np.random.default_rng(int(hashlib.md5(key.encode()).hexdigest()[:8], 16))

def _pick(rng, opts):
    return opts[rng.integers(0, len(opts))]

# Phrase banks (English; customer voice)
CHANNELS = ["App Chat", "Call Center", "Email", "Whatsapp", "Web Form"]
TOPICS   = ["Connectivity", "Speed", "Intermittent", "Outage", "Billing", "Installation", "General"]
OPENERS  = [
    "Hi, I'm having trouble with my connection.",
    "Hello, I need help with my internet.",
    "Good day, my service isn't working as expected.",
    "Hi team, my connection is unstable.",
    "Hello, I’d like to report a service issue."
]
SYMPTOMS = [
    "frequent disconnections",
    "slow speed on video calls",
    "web pages timing out",
    "packet loss during gaming",
    "streaming keeps buffering",
    "VPN drops every few minutes",
    "high latency spikes"
]
EMPHASIS = [
    "This started recently and it’s getting worse.",
    "It’s been happening on and off for hours.",
    "It’s affecting work calls and meetings.",
    "It’s been stable before; today it’s bad.",
    "I already restarted the router with no improvement."
]
ASKS = [
    "Can you check and fix this?",
    "Please advise what I should do next.",
    "Is there an outage in my area?",
    "Can you send a technician if needed?",
    "Please let me know the ETA for a fix."
]
THANKS = [
    "Thanks.",
    "Thank you.",
    "Appreciate your help.",
    "Looking forward to your support.",
    "Regards."
]
PRAISE = [
    "Service has been very stable lately.",
    "Speed looks great and consistent.",
    "Everything works fine today.",
    "No issues to report; happy with the service."
]
BILLING = [
    "I was charged a late fee unexpectedly.",
    "My bill seems higher than usual.",
    "I already paid but the app still shows overdue.",
    "Payment went through but service was limited."
]

def _derive_feedback(row):
    rng = _rng_for_row(row)

    # Signals from the customer table
    dev_status   = row.get("device_status", "Online")
    link_state   = row.get("device_link_os", "Up")
    reason       = row.get("offline_reason", "Unknown")
    last_offline = row.get("last_offline_ts", pd.NaT)
    last_online  = row.get("last_online_ts", pd.NaT)
    sat          = float(row.get("user_satisfaction_score", 0.8))
    comp_mo      = int(row.get("complaints_this_month", 0))
    usage_gb     = float(row.get("network_usage_gb", 0.0))
    late_pays    = int(row.get("late_payment_count", 0))
    churn        = int(row.get("churn_flag", 0))

    # Recentness flags (if last_offline_ts is within ~48h of last_online_ts)
    recent_issue = False
    try:
        if pd.notna(last_offline) and pd.notna(last_online):
            recent_issue = (last_online - last_offline) <= pd.Timedelta(hours=48)
    except Exception:
        pass

    # Severity heuristic (customer perspective)
    hard_down   = (dev_status == "Offline") or (link_state == "Down")
    flapping    = (link_state == "Flapping")
    degrade_hint= recent_issue or flapping or (reason in ["Power outage","Fiber issue"])
    low_sat     = sat < 0.75
    heavy_user  = usage_gb > 200

    sev = (
        2.0*hard_down +
        1.2*flapping +
        1.0*degrade_hint +
        0.6*(comp_mo >= 5) +
        0.4*(heavy_user)
    )

    # Pick channel/topic
    channel = _pick(rng, CHANNELS)
    topic   = "Connectivity"

    # Billing detour if late payments + not severe tech issue
    billing_bias = (late_pays > 0) and (sev < 1.2) and rng.random() < 0.35
    if billing_bias:
        topic = "Billing"

    # Sentiment & rating mapping
    if topic == "Billing":
        sentiment = "negative" if late_pays > 0 else "neutral"
        rating = 2 if late_pays > 0 else 3
    else:
        if sev >= 2.2:
            sentiment, rating = "negative", 1
        elif sev >= 1.2:
            sentiment, rating = "negative", 2
        elif sev >= 0.6:
            sentiment, rating = "neutral", 3
        else:
            sentiment, rating = ("positive" if sat >= 0.9 and comp_mo <= 1 else "neutral"), (5 if sat >= 0.9 else 4)

    # Build text (customer voice)
    city = row.get("city","")
    cluster = row.get("cluster_name","")
    loc_hint = f"in {city} ({cluster})" if city and cluster else ""
    opener = _pick(rng, OPENERS)

    if topic == "Billing":
        body = _pick(rng, BILLING)
        tail = _pick(rng, THANKS)
        text = f"{opener} {body} {tail}"
    else:
        # Technical body
        sym = _pick(rng, SYMPTOMS)
        emph = _pick(rng, EMPHASIS)

        # Insert concrete cues
        cues = []
        if hard_down:
            cues.append("the modem shows no internet")
        if flapping:
            cues.append("the connection keeps dropping and coming back")
        if reason != "Unknown" and (hard_down or degrade_hint):
            cues.append(f"your app mentions '{reason.lower()}'")
        if heavy_user and rng.random() < 0.4:
            cues.append("I rely on this for large file uploads")

        cue_text = (" Also, " + "; ".join(cues) + ".") if cues else ""
        last_seen = ""
        if pd.notna(last_offline) and rng.random() < 0.6:
            last_seen = f" Last time it went down was around {pd.to_datetime(last_offline).strftime('%Y-%m-%d %H:%M')}."
        ask = _pick(rng, ASKS)
        tail = _pick(rng, THANKS)

        text = (
            f"{opener} I'm experiencing {sym} {loc_hint}. {emph}{cue_text}"
            f"{last_seen} {ask} {tail}"
        )

    # Stable feedback_id for downstream search/tracking
    base = f"{row.get('customer_id','?')}|{row.get('cluster_id','?')}|{row.get('last_offline_ts','?')}"
    feedback_id = "FB-" + hashlib.md5(base.encode()).hexdigest()[:16].upper()

    return pd.Series({
        "feedback_id": feedback_id,
        "feedback_channel": channel,
        "feedback_topic": topic,
        "sentiment_label": sentiment,
        "rating_1_5": int(rating),
        "feedback_text": text
    })

# Build and attach columns
new_cols = fb.apply(_derive_feedback, axis=1)
fb_out = pd.concat([fb, new_cols], axis=1)

# Save as a NEW artifact (non-destructive)
fb_out.to_parquet(out_path, index=False)

print("Customer feedback text added.")
print("Saved:", out_path)
print("Columns added: feedback_id, feedback_channel, feedback_topic, sentiment_label, rating_1_5, feedback_text")

Customer feedback text added.
Saved: dataset/customer_feedback_text/ews_customer_feedback_with_text.parquet
Columns added: feedback_id, feedback_channel, feedback_topic, sentiment_label, rating_1_5, feedback_text


---

In [12]:
df = pd.read_parquet("dataset/customer_feedback_text/ews_customer_feedback_with_text.parquet")

In [13]:
pd.set_option('display.max_colwidth', None)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   city                     204 non-null    object        
 1   city_key                 204 non-null    int64         
 2   province                 204 non-null    object        
 3   region                   204 non-null    object        
 4   olt_id                   204 non-null    object        
 5   olt_key                  204 non-null    int64         
 6   cluster_id               204 non-null    object        
 7   cluster_key              204 non-null    int64         
 8   cluster_name             204 non-null    object        
 9   customer_id              204 non-null    object        
 10  home_pass                204 non-null    object        
 11  home_connect_id          204 non-null    object        
 12  join_date                204 non-nul

In [16]:
df.head(5)

,city,city_key,province,region,olt_id,olt_key,cluster_id,cluster_key,cluster_name,customer_id,...,device_link_os,last_online_ts,last_offline_ts,offline_reason,feedback_id,feedback_channel,feedback_topic,sentiment_label,rating_1_5,feedback_text
0,Aceh,1,Aceh,Sumatra,OLT-ACE7E-01,1,ACE7E-CL11,1,Cluster-ACE7E-11,CUST00000,...,Up,2024-12-17 23:00:00,2024-12-17 16:00:00,Fiber issue,FB-E3DA1C7A22068D61,Whatsapp,Connectivity,neutral,3,"Hi team, my connection is unstable. I'm experiencing packet loss during gaming in Aceh (Cluster-ACE7E-11). It’s been stable before; today it’s bad. Also, your app mentions 'fiber issue'. Can you check and fix this? Thank you."
1,Aceh,1,Aceh,Sumatra,OLT-ACE7E-01,1,ACE7E-CL12,2,Cluster-ACE7E-12,CUST00001,...,Up,2024-12-26 09:00:00,2024-12-24 16:00:00,Unknown,FB-704D72388DDE07BD,Web Form,Connectivity,neutral,3,"Hi team, my connection is unstable. I'm experiencing high latency spikes in Aceh (Cluster-ACE7E-12). It’s affecting work calls and meetings. Last time it went down was around 2024-12-24 16:00. Can you check and fix this? Looking forward to your support."
2,Aceh,1,Aceh,Sumatra,OLT-ACE7E-01,1,ACE7E-CL13,3,Cluster-ACE7E-13,CUST00002,...,Up,2024-12-29 17:00:00,2024-12-28 15:00:00,Unknown,FB-4472C720EDCAC9F6,Web Form,Billing,negative,2,"Hello, I need help with my internet. My bill seems higher than usual. Appreciate your help."
3,Aceh,1,Aceh,Sumatra,OLT-ACE7E-02,2,ACE7E-CL21,4,Cluster-ACE7E-21,CUST00003,...,Down,2024-12-22 09:00:00,2024-12-19 19:00:00,Unknown,FB-9D9BA724F8336BB8,Web Form,Connectivity,negative,2,"Hi team, my connection is unstable. I'm experiencing slow speed on video calls in Aceh (Cluster-ACE7E-21). It’s affecting work calls and meetings. Also, the modem shows no internet. Last time it went down was around 2024-12-19 19:00. Can you check and fix this? Thank you."
4,Aceh,1,Aceh,Sumatra,OLT-ACE7E-02,2,ACE7E-CL22,5,Cluster-ACE7E-22,CUST00004,...,Up,2024-12-21 06:00:00,2024-12-18 12:00:00,Unknown,FB-7313705082AB26F6,Whatsapp,Billing,negative,2,"Hi, I'm having trouble with my connection. I already paid but the app still shows overdue. Thanks."
